In [1]:
import cobra
import cobra.core
from cobra.core import Model, Reaction, Metabolite
from IPython.display import Image

![title](img/toy_model.png)

Figura 2. Modelo metabólico de juguete compuesto por tres metabolito (A, B y C), cuatro reacciones internas
(v1-v4) y tres flujos de intercambio (b1-b3). a) representación gráfica del modelo; b) representación matricial de
las ecuaciones de balance. N tambien llamada S es la matriz estequiométrica.

In [2]:
model = Model('Toymodel')

In [3]:
A = Metabolite("A")
A.compartment = 'cytosol'
B = Metabolite("B")
B.compartment = 'cytosol'

### Excercise 1.1

Create Metabolite C


In [4]:
######################
# Create Metabolite C
######################
C = Metabolite("C")

In [5]:
## Add the metabolites to the model
model.add_metabolites([A, B, C])

In [6]:
# Print the reactions
print(model.metabolites.A.reactions)

frozenset()


In [7]:
# Creating the reactions

# Create reaction with id b1
b1 = Reaction("b1")

# To add metabolites to the reaction passing
# a dictionary withs metabolites as the keys
# and the stoichiometric coefficients as values
b1.add_metabolites({A: 1})

# the same is done for the other reactions
b2 = Reaction("b2")
b2.add_metabolites({B: -1})

b3 = Reaction("b3")
b3.add_metabolites({C: -1})

v1 = Reaction("v1")
v1.add_metabolites({A:-1, B:1})

v2 = Reaction("v2")
v2.add_metabolites({A:-1, C:1})

v3 = Reaction("v3")
v3.add_metabolites({A:1, C:-1})

# Create v4 (Excercise 1.2)

### Excercice 1.2: 

Create reactions v4 (and add its metabolites

In [8]:
########################
# Create reactions v4
########################

v4 = Reaction("v4")
v4.add_metabolites({B:1, C:-1})

In [9]:
# Adding the reactions to de toy model

model.add_reactions([b1,b2,b3,v1,v2,v3,v4])

### Write the model in SBML format

1. First import the corresponding function
2. write the model
3. Optional you can inspecte the SBML using some plain text editor.

In [10]:
import cobra.io
from cobra.io import write_sbml_model

# Saving the model to a file in sbml format
write_sbml_model(model, "out/toymodel.sbml")

### Optimización – calculo de una distribución de flujos óptima
El problema es encontrar encontrar un conjunto de valor de flujo (o velocidades) para cada reacción de la red, que sea compatible con las restricciones del análisis estequiométrico (i.e. estado estacionario, reversibilidades y cotas) y que sea óptima (maxima o minimal) respecto a algún objetivo definido.

In [11]:
# Antes de optimizar debemos establecer la condiciones de crecimiento
# es decir las velocidades máximas de los flujos de entrada
model.reactions.b1.upper_bound = 1

# Luego indicamos la velocidad que deseamos optimizar (i.e. nuestro 
# objetivo) que para nuestro caso será optimizar el flujo de b2
# Para esto asignamos 1 al coeficiente objetivo de b2
model.reactions.b2.objective_coefficient = 1

# To compute and FBA on the model we use the following function:
solution = model.optimize()

# the results is a solution object which contains
solution.fluxes.to_csv("out/toymodel_fba.tsv", sep="\t")

# Para ver la solución  y como valor el flujo de dicha reacción en la soluión
# optima encontrada
# solution.fluxes

b1    1.0
b2    1.0
b3    0.0
v1    1.0
v2    0.0
v3    0.0
v4    0.0
Name: fluxes, dtype: float64

# Parte 2 - Constraint-based modeling at genome-scale

En esta sección emplearemos un modelo metabólico de E. coli reconstruido a escala genómica denominado iJO1366.

2. Lectura de fichero SBML e inspección general del modelo

importar cobra.io si no fue importada (ver 1.2).
Lectura del fichero SBML (modelo metabólico a escala genómica de Escherichia coli). Para esto deberán tener descargado el fichero iJO1366.xml

In [12]:
from cobra.io import read_sbml_model

# Ruta a donde se hayan descargado el archivo iJO1366.xml)
sbml_fname = './data/iJO1366.xml'

# lectura del model desde un fichero habiendo importado previamente la función
model = read_sbml_model(sbml_fname)

### Inspección del Modelo (Reacciones)

Para verificar que el modelo ha sido cargado de forma correcta podéis probar el siguiente comando: print model

El ordenador debe imprimir en pantalla: iJO1366
Extraer propiedades generales del modelo empleando las siguiente líneas:
1. Listar reacciones: model.reactions (estos es una lista, usar un for con prints)
2. Número total de reacciones: print len(model.reactions)
3. Seleccionar varias reacciones y ver: su nombre, formula, gen/es codificante/s, reversibilidad y las cotas (upper y lower bounds). Por ejemplo para la NADH16pp (HINT: la tecla tab autocompleta y muestra opciones)

<br>
Comparar la reacciones con su descripción en Ecocyc:
<br>
http://ecocyc.org/ECOLI/NEW-IMAGE?type=NIL&object=EG12084&redirect=T

In [13]:
### TODO



### Inspección del Modelo (Metaboitos)

* Listar metabolitos: model.metabolites
* Número total de metabolitos: print len(model.metabolites)
* Seleccionar varios metabolitos y ver: su nombre, el compartimiento y las reacciones en las que participa. Por ejemplo para la g6p_c:



In [14]:
### TODO





## Parte 2.2 Cálculo del modo de flujos óptimo, utilizando Flux Balance Analysis (FBA)

1. Realizar la optimización (maximizar) la producción de biomasa. 
2. La condición de borde que el modelo tiene por defecto, permite una entrada ilimitada de oxigeno pero hay un limite en la entrada de glucosa fijado en -10. 
3. Verificar lo anterior mirando los atributos lower_bound de las reacciones de intercambio: EX\_glc\_e\_ y EX\_o2\_e\_.

In [15]:
### TODO




### Identificar en el listado generado anteriormente: 
* el valor que toman los flujos de intercambio de la glucosa (EX_glc_e_) y el oxígeno (EX_o2_e_) en la solución óptima calculada
* compararlo con el valor de los lower_bound de ambas reacciones.

HINT: usar el objeto solución -> solution.fluxes.reaction_id

In [16]:
## TODO




### 2.3 - Visualizando Modos de Flujo Óptimos
Para poder realizar esta parte vamos a emplear una herramienta de visualización web Pero esta herramienta requiere que le demos un fichero con las reacciones del modelo y los valores de los flujos de la solución que deseamos visualizar. De forma que hace falta guardar los resultados en un fichero de texto en formato json y que nos descarguemos el fichero a nuestro ordenador personal.

Para facilitar la visualización vamos a emplear un método alternativo de 
optimización que se llama pFBA:
* pFBA Basicamente es un FBA que luego elimina todos los ciclos que aparezcan en una solución
* Para esto se debe importar la función pfba que se encuentra en el módulo:

from cobra.flux_analysis import pfba
<br>
Una vez salvada la solución a formato json visualizar la distribución en la herramienta escher:
* http://escher.github.io/


In [17]:
from cobra.flux_analysis import pfba
import json

# luego guardamos la solución en una variable de tipo diccionario
wt_solution = pfba(model)
solution_dict = dict(wt_solution.fluxes)

# abrimos un fichero de nombre wt_solution.json en modo escritura
# escribimos el en el fichero y cerramos el fichero 
with open('wt_pfba_solution.json','w') as fh:
    json.dump(solution_dict, fh)

## Parte 3

### 3.1 – Knockout in silico
Un experimento de knockout in silico consiste en fijar a cero el flujo a través de todas las reacciones que cataliza el gen para el cual estamos simulando el knockout. 

Ejemplo 1: el gen que codifica para la actividad Citrato Sintasa es el gen b0720
Para poder ver dicha asociación basta ver los siguiente:

In [18]:
# seleccionamos el gene
gene =  model.genes.b0720
# seleccionamos las reacciones asociadas a dicho gen
# imprimimos las reacciones asociadas a b0720
for r in gene.reactions: 
    print("ID:%s | Nombre: %s" % (r.id,r.name))

# Lo mismo comenzando desde la reacción.
reaction = model.reactions.CS
for g in reaction.genes: 
    print("ID:%s" % g.id)

ID:CS | Nombre: citrate synthase
ID:b0720


### Ejercicio 3.1

Como vimos en en la teoría
* simular un knockout in-silico de un gen equivale a fijar a 0 las cotas de la/las reacciones que codifique 
* para lo cual utilizamos los atributos lower_bound y  upper_bound
* a continuación optimizamos -> model.optimize()
* finalmente evaluamos la velocidad de crecimiento del mutante

In [19]:
## TODO

# Entonces simular un knockout in silico de b0720 equivale a fijar a 0 la reacción CS para lo cual utilizamos 
# los atributos upper_bound (ya que el lower_bound es 0 porque la reacción es irreversible):


# Buscamos el modo de flujo que maximiza el crecimiento



# miramos el valor del crecimiento predicho, que sucede?
# Hint: solution.fluxes.Ec_biomass_iJO1366_WT_53p95M




¿El experimento in silico que nos indica? ¿El knockout para b0720 es letal?
<br>
Compara  con el resultado in vivo en esta base de datos:
* https://ecocyc.org/gene?orgid=ECOLI&id=EG10402

# 3.2 – Knockout in silico: Large Experiment
    
* Cobra tiene implementada una función para realizar los experimentos de knockout. 
* En esta sección emplearemos dicha función para realizar un experimento de knockout sobre todos los genes que incluye el modelo y esto lo compararemos con resultados in vivo
* Primero se requiere cargar los siguiente módulos:

In [20]:
from cobra.flux_analysis import single_gene_deletion

# volvemos a cargar el modelo ya que lo hemos modificado
model = read_sbml_model('./data/iJO1366.xml')

threshold = 0.05

In [21]:
# experimento in-silico (tarda unos minutos)
knockout = single_gene_deletion(model)
index_mapper = {i:list(i)[0] for i in knockout.index}
knockout = knockout.rename(mapper=index_mapper, axis=0)

In [22]:
# Lectura de la lista de genes letales in vivo
import json
fname = './data/m9_invivo_lethals.json'
with open(fname) as fh:
    invivo_lethals = json.load(fh)
    invivo_lethals = set(invivo_lethals)

# crear un “set” con los nombres de todos los genes del modelo
all_genes = set([g.id for g in model.genes])

# aplicamos una operación de conjuntos (set) para obtener los genes no 
# letales in-vivo
invivo_non_lethals = all_genes - invivo_lethals
print("invivo_non_lethals:", len(invivo_non_lethals))
print("invivo_lethals:", len(invivo_lethals))

# Obtención de no letales in silico que son los que si predicen crecimiento
insilico_non_lethals = set(knockout.index[knockout.growth > threshold])
print("insilico_non_lethals:", len(insilico_non_lethals))

# Obtención de letales in silico filtrando lo genes cuyo knockout causo una # caída en la producción de biomasa por debajo de un umbrar (< 5% del optimo)
insilico_lethals = set(knockout.index[knockout.growth< threshold])
print("insilico_lethals:", len(insilico_lethals))

invivo_non_lethals: 1129
invivo_lethals: 238
insilico_non_lethals: 1106
insilico_lethals: 261


In [23]:
# https://en.wikipedia.org/wiki/Receiver_operating_characteristic

# El conjunto de True Negatives, son aquellos genes predichos como NO Escenciales
# y el dato experimental indica que el gene es essencial en las condiciones de estudio
TN = insilico_non_lethals & invivo_non_lethals

# El conjunto de True Negatives, son aquellos genes predichos como Escenciales
# y el dato experimental indica que el gene es essencial en las condiciones de estudio
TP =  insilico_lethals & invivo_lethals

# Los False Negatives, son aquellos genes predichos como NO Escenciales
# pero que el dato experimental indica que el gene es essencial en las condiciones de estudio
FN = insilico_non_lethals & invivo_lethals

# Los False positives, son aquellos genes predichos como Escenciales
# pero que el dato experimental indica que el gene NO es essencial en las condiciones de estudio
FP = insilico_lethals & invivo_non_lethals

P = TP | FN
N = TN | FP

In [25]:
# https://en.wikipedia.org/wiki/Receiver_operating_characteristic

# sensitivity, recall, hit rate, or true positive rate (TPR)
sensitivity = len(TP) / len(P) 

# specificity, selectivity or true negative rate (TNR)
specificity = len(TN) / len(N)

# precision or positive predictive value (PPV)
precision = len(TP) / (len(TP) + len(FP)) 

# accuracy (ACC)
accuracy = (len(TP) + len(TN)) / (len(P) + len(N))

0.8749085588880761